In [1]:
import torch
from torch.utils.data import  DataLoader
from torchvision.models import densenet201, DenseNet201_Weights
from torchvision.datasets import ImageFolder

from torch.optim import  AdamW
from torch import nn


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [3]:
dataset = ImageFolder("./dataset/", transform=DenseNet201_Weights.IMAGENET1K_V1.transforms())

In [4]:
dataset.classes

['adobo', 'inasal_na_manok', 'sinigang', 'sisig', 'tortang_talong']

In [5]:
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [6]:
vision_model = densenet201(weights=DenseNet201_Weights.IMAGENET1K_V1)
vision_model.classifier = nn.Linear(vision_model.classifier.in_features, 5)
vision_model = vision_model.to(device)

In [7]:
optimizer = AdamW(vision_model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [8]:
epoch = 20
for e in range(epoch):
    vision_model.train()
    for i, (img, label) in enumerate(dataloader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        y_pred = vision_model(img)
        loss = criterion(y_pred, label)

        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoc: {e}, Step: {i}, loss: {loss.item():.4f}")


Epoc: 0, Step: 0, loss: 1.6202
Epoc: 0, Step: 10, loss: 0.9668
Epoc: 0, Step: 20, loss: 0.6742
Epoc: 0, Step: 30, loss: 0.3655
Epoc: 0, Step: 40, loss: 0.1265
Epoc: 0, Step: 50, loss: 0.1020
Epoc: 0, Step: 60, loss: 0.6183
Epoc: 1, Step: 0, loss: 0.0782
Epoc: 1, Step: 10, loss: 0.2147
Epoc: 1, Step: 20, loss: 0.0299
Epoc: 1, Step: 30, loss: 0.0547
Epoc: 1, Step: 40, loss: 0.0696
Epoc: 1, Step: 50, loss: 0.1879
Epoc: 1, Step: 60, loss: 0.0172
Epoc: 2, Step: 0, loss: 0.0152
Epoc: 2, Step: 10, loss: 0.1451
Epoc: 2, Step: 20, loss: 0.0943
Epoc: 2, Step: 30, loss: 0.0247
Epoc: 2, Step: 40, loss: 0.0729
Epoc: 2, Step: 50, loss: 0.0128
Epoc: 2, Step: 60, loss: 0.0468
Epoc: 3, Step: 0, loss: 0.1051
Epoc: 3, Step: 10, loss: 0.0976
Epoc: 3, Step: 20, loss: 0.0766
Epoc: 3, Step: 30, loss: 0.0746
Epoc: 3, Step: 40, loss: 0.0781
Epoc: 3, Step: 50, loss: 0.1992
Epoc: 3, Step: 60, loss: 0.0312
Epoc: 4, Step: 0, loss: 0.0235
Epoc: 4, Step: 10, loss: 0.0968
Epoc: 4, Step: 20, loss: 0.0191
Epoc: 4, Step

In [9]:
from torchmetrics.classification import MulticlassF1Score

f1_metric = MulticlassF1Score(num_classes=5, average='macro').to(device)

In [10]:
vision_model.eval()
with torch.no_grad():
    for i, (img, label) in enumerate(dataloader):
        img = img.to(device)
        label = label.to(device)
        y_pred = vision_model(img)
        y_pred_class = torch.argmax(y_pred, dim=1)
        f1_metric.update(y_pred_class, label)

print(f1_metric.compute())

tensor(1., device='cuda:0')


In [13]:
torch.save(vision_model.state_dict(), f"./model.pt")

In [ ]:
all_preds = []
all_labels = []

vision_model.eval()
with torch.no_grad():
    for img, label in dataloader:
        img = img.to(device)
        label = label.to(device)

        y_pred = vision_model(img)
        y_pred_class = torch.argmax(y_pred, dim=1)

        all_preds.append(y_pred_class.cpu())
        all_labels.append(label.cpu())

# Stack and analyze
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

# Print unique prediction distribution
unique_preds, counts = all_preds.unique(return_counts=True)
print("Predicted class distribution:", dict(zip(unique_preds.tolist(), counts.tolist())))

# Print label distribution too
unique_labels, label_counts = all_labels.unique(return_counts=True)
print("True label distribution:", dict(zip(unique_labels.tolist(), label_counts.tolist())))

# Sample predicted and true values
print("Sample predictions:", all_preds[:10].tolist())
print("Sample labels:     ", all_labels[:10].tolist())


🔢 Predicted class distribution: {0: 101, 1: 100, 2: 101, 3: 100, 4: 100}
🏷️ True label distribution: {0: 101, 1: 100, 2: 101, 3: 100, 4: 100}
📌 Sample predictions: [3, 0, 1, 1, 4, 1, 4, 3, 3, 3]
📌 Sample labels:      [3, 0, 1, 1, 4, 1, 4, 3, 3, 3]
